In [24]:
import os
from pathlib import Path

# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/Users/liuqiaowei/SourceCode/freqtrade/user_data/notebooks"
i = 0
try:
    os.chdir(project_root)
    if not Path("LICENSE").is_file():
        i = 0
        while i < 4 and (not Path("LICENSE").is_file()):
            os.chdir(Path(Path.cwd(), "../"))
            i += 1
        project_root = Path.cwd()
except FileNotFoundError:
    print("Please define the project root relative to the current directory")
print(Path.cwd())

/Users/liuqiaowei/SourceCode/freqtrade


In [25]:
from freqtrade.configuration import Configuration


# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["./user_data/config.json"])
# Optionally (recommended), use existing configuration file
# config = Configuration.from_files(["user_data/config.json"])

# Location of the data
data_location = config["datadir"]
config['timeframe'] = '5m'
# Pair to analyze - Only use one pair here
# config['exchange']['pair_whitelist']

In [28]:
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from pandas_ta import percent_return
import pandas_ta as ta

candles_dict = {}
for pair in config['exchange']['pair_whitelist']:
    candles = load_pair_history(
        datadir=data_location,
        timeframe=config["timeframe"],
        pair=pair,
        data_format="feather",  # Make sure to update this to your data
        candle_type=CandleType.FUTURES,
    )
    candles['sym'] = pair
    candles.set_index('date', inplace=True)
    candles.set_index('sym', append=True, inplace=True)
    candles['return'] = percent_return(candles.close, offset=-12)
    
    
    candles_dict[pair] = candles

In [41]:
import numpy as np
pattern_long_return_sum, pattern_short_return_sum = 0, 0

for key, value in candles_dict.items():
    candles = value.copy()
    candles.dropna(inplace=True)
    candles['3linestrike'] = candles.ta.cdl_pattern(name="3linestrike")
    candles_long = candles[candles['3linestrike'] == 100]
    candles_short = candles[candles['3linestrike'] == -100]
    
    pattern_long_return_sum += candles_long['return'].mean() if candles_long['return'].mean() != np.nan else 0
    pattern_short_return_sum += candles_short['return'].mean() if candles_short['return'].mean() != np.nan else 0
    
    print(f"{key}: 3inside long count {candles_long.shape[0]} return mean: {candles_long['return'].mean()}, \n "
          f"3inside short count {candles_short.shape[0]} short mean: {candles_short['return'].mean()}")
    

BTC/USDT:USDT: 3inside long count 75 return mean: -0.00022703968399531593, 
 3inside short count 77 short mean: -0.00018632761364537604
ETH/USDT:USDT: 3inside long count 67 return mean: 5.805998308772816e-05, 
 3inside short count 85 short mean: -5.5299133018489846e-05
BNB/USDT:USDT: 3inside long count 83 return mean: 0.00031933916615152643, 
 3inside short count 71 short mean: 8.088889856908496e-05
XRP/USDT:USDT: 3inside long count 56 return mean: 0.0002690430239366152, 
 3inside short count 30 short mean: 0.001545259537909486
ADA/USDT:USDT: 3inside long count 50 return mean: 7.837453649784942e-05, 
 3inside short count 22 short mean: 0.00038388489329933953
DOGE/USDT:USDT: 3inside long count 68 return mean: -0.00014544741602996, 
 3inside short count 62 short mean: 9.703338048897809e-06
SOL/USDT:USDT: 3inside long count 102 return mean: -3.1229887319961236e-05, 
 3inside short count 97 short mean: -0.00018814257794873523
TRX/USDT:USDT: 3inside long count 67 return mean: -5.76081380968

In [42]:
pattern_long_return_sum / len(candles_dict.keys())

nan

In [45]:
pattern_short_return_sum / len(candles_dict.keys())

nan